Import Libraries

In [32]:
import pandas as pd
import numpy as np
import string
import spacy
spacy.load("en_core_web_sm")
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

Loading Data

In [33]:

df_train = pd.read_csv('datasets_897_1638_Sheet_2.csv', header=0, encoding = 'unicode_escape')
df_train.head()

,resume_id,class,resume_text
0,resume_1,not_flagged,\rCustomer Service Supervisor/Tier - Isabella ...
1,resume_2,not_flagged,\rEngineer / Scientist - IBM Microelectronics ...
2,resume_3,not_flagged,\rLTS Software Engineer Computational Lithogra...
3,resume_4,not_flagged,TUTOR\rWilliston VT - Email me on Indeed: ind...
4,resume_5,flagged,\rIndependent Consultant - Self-employed\rBurl...


In [34]:
y = df_train['class']

y = np.array(list(map(lambda x: 1 if x=="flagged" else 0, y)))

In [35]:
y

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0])

shape of dataframe

In [36]:
df_train.shape

(125, 3)

View data information

In [37]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   resume_id    125 non-null    object
 1   class        125 non-null    object
 2   resume_text  125 non-null    object
dtypes: object(3)
memory usage: 3.1+ KB


Tokening the data with SpaCy

In [38]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

Create our list of punctuation marks

In [39]:
punctuations = string.punctuation

Create our list of stopwords

In [40]:
#nlp = spacy.load('en')
#stop_words = spacy.lang.en.stop_words.STOP_WORDS

nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

/home/chi/anaconda3/envs/nltkEnv/lib/python3.7/site-packages/spacy/util.py:271: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Load English tokenizer, tagger, parser, NER and word vectors

In [41]:
parser = English()

Creating our tokenizer function

In [42]:
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PORN-" else word.lower_ for word in mytokens]
    #mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]
    
    return mytokens

Defining a custom Transformer

In [44]:
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    
    def fit(self, X, y = None, **fit_params):
        return self
    
    def get_params(self, deep = True):
        return {}
    
    
    
    
#Basic function to clean the text
def clean_text(text):
    return text.strip().lower()

Vectorization feature engineering(TF-IDF)

In [45]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range = (1,1))

Splitting the data into training and test sets

In [46]:
from sklearn.model_selection import train_test_split
X = df_train['resume_text']
ylabels = y

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size = 0.3)

Creating a pipeline and Generating the model

Logistic Regression Classifier

In [47]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

Create pipeline using Bag of Words

In [49]:
pipe = Pipeline([("cleaner", predictors()),
                ('vectorizer', bow_vector),
                ('classifier', classifier)])

Model generation

In [50]:
pipe.fit(X_train, y_train)

/home/chi/anaconda3/envs/nltkEnv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7f1a24b35310>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 t...\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x7f1a24a4c050>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
            

Evaluating the model

In [51]:
from sklearn import metrics
#Predicting with a test dataset
predicted = pipe.predict(X_test)

#
#print()

Model Accuracy

In [53]:
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_test, predicted))

print("Logistic Regression Precision:", metrics.precision_score(y_test, predicted))

print("Logistic Regression Recall:", metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.7368421052631579
Logistic Regression Precision: 0.3333333333333333
Logistic Regression Recall: 0.25
